In [1]:
# import dependencies

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas import DataFrame

In [2]:
html = "https://www.zip-codes.com/county/ca-san-diego.asp"
contents = requests.get(html)

soup = BeautifulSoup(contents.content, 'lxml')
# Need to parse through DOM to know which table to scrape
table = soup.find_all('table')[2]
lists = pd.read_html(str(table))

In [3]:
website_data_df = lists[0]
print(f'Initial size of dataframe: {website_data_df.shape}')
# Set column headers
website_data_df.columns = website_data_df.iloc[0]
website_data_df = website_data_df.drop(0)
# Reset index 
website_data_df = website_data_df.reset_index(drop=True)
website_data_df.head()

Initial size of dataframe: (182, 6)


,ZIP Code,Classification,City,Population,Timezone,Area Code(s)
0,ZIP Code 91901,General,Alpine,17403,Pacific,619
1,ZIP Code 91902,General,Bonita,17653,Pacific,619
2,ZIP Code 91903,P.O. Box,Alpine,0,Pacific,619
3,ZIP Code 91905,General,Boulevard,1700,Pacific,619/760
4,ZIP Code 91906,General,Campo,3627,Pacific,619


In [4]:
# Drop Zip Codes with Population 0
no_population = website_data_df[website_data_df["Population"] == '0'].index
website_data_df.drop(no_population, inplace=True)
print(f'Updated size of dataframe: {website_data_df.shape[0]} rows and {website_data_df.shape[1]} columns')
# Drop "Zip Code" phrase in Zip Code column
website_data_df["ZIP Code"] = website_data_df["ZIP Code"].str[-5:]
website_data_df.head()

Updated size of dataframe: 106 rows and 6 columns


,ZIP Code,Classification,City,Population,Timezone,Area Code(s)
0,91901,General,Alpine,17403,Pacific,619
1,91902,General,Bonita,17653,Pacific,619
3,91905,General,Boulevard,1700,Pacific,619/760
4,91906,General,Campo,3627,Pacific,619
7,91910,General,Chula Vista,75802,Pacific,619


In [8]:
import geopy
from geopy.geocoders import  Nominatim

geolocator = Nominatim(user_agent="sd_explorer")


lats = []
longs = []

for zip in website_data_df["ZIP Code"]:
    address = zip
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    lats.append(latitude)
    longs.append(longitude)



In [12]:
website_data_df["Latitude"] = lats
website_data_df["Longitude"] = longs

In [13]:
website_data_df.head()

,ZIP Code,Classification,City,Population,Timezone,Area Code(s),Latitude,Longitude
0,91901,General,Alpine,17403,Pacific,619,48.414003,17.491107
1,91902,General,Bonita,17653,Pacific,619,32.666379,-117.022725
3,91905,General,Boulevard,1700,Pacific,619/760,32.662786,-116.312998
4,91906,General,Campo,3627,Pacific,619,32.662692,-116.481502
7,91910,General,Chula Vista,75802,Pacific,619,48.534285,2.181133


In [14]:
# Install and import folium as map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage from SciKit-Learn

from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

In [16]:
sd_lat = 32.715736
sd_long = -117.161087

map_sd = folium.Map(location=[sd_lat, sd_long], zoom_start = 11)

# add markers to map

map_sd